In [3]:
import pandas as pd
from hugchat import hugchat
from hugchat.login import Login
import json
from tqdm import tqdm
tqdm.pandas()

In [4]:
# Load the secrets from the JSON file
with open('client_secrets.json', 'r') as file:
    secrets = json.load(file)

# Set up the API key and YouTube API client
login = secrets['huggingLogin']
password = secrets['huggingPassword']

### Initialize HuggingChat

In [5]:
sign = Login(login, password)
cookies = sign.login()

# Save cookies to the local directory
cookie_path_dir = "./cookies_snapshot"
sign.saveCookiesToDir(cookie_path_dir)

'./cookies_snapshot/accounts@gregordebus.de.json'

In [6]:
# Load cookies when you restart your program:
# sign = login(email, None)
# cookies = sign.loadCookiesFromDir(cookie_path_dir) # This will detect if the JSON file exists, return cookies if it does and raise an Exception if it's not.

In [7]:
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())  # or cookie_path="usercookies/<email>.json"

### Load Shorts Information

In [8]:
df_shorts = pd.read_csv("youtube_shorts_description_gdebus.csv", sep=";")
df_shorts.head()

,Video ID,Video Title,Channel Title,Transcript,Duration,Words per Second,Number of Comments,Top10 Comments,Category,Views,Likes
0,6TYFH8UXGm4,b'CRAZIEST Sagawa1gou Funny TikTok Compilation...,b'Sagawa /\xe3\x81\x95\xe3\x81\x8c\xe3\x82\x8f',foreign [Music] foreign [Music] [Music] [Music...,1406.033,0.359167,804,['This guy can&#39;t fail to make us laugh 😂 a...,b'Entertainment',8087085,96939
1,iye3itnH9cs,b'Watch this and try not to laugh \xf0\x9f\x98...,b'Fruity Vibes',[Laughter] nice uh nice view out there with [A...,20.358,0.589449,4279,['&quot;Our videos are love notes from us to y...,b'People & Blogs',31973780,743037
2,n_6YPvWcFLA,b'Funny baby reaction on the beach || #shorts',b'Fun Decade',[Music] all right careful [Music],25.730,0.233191,4145,['onça com a tarde de mim não TJRJ nada de mai...,b'Comedy',223683308,3022620
3,7bnKr6XfcWc,b'Funny Moment #shorts #funny',b'BoomBibo',oh my God oh yeah foreign,9.357,0.748103,11446,"['If you think kids are immature, remember thi...",b'Comedy',222227209,3074313
4,qXxC0R9CQkI,b'the gay test #shorts #comedy #funny',b'al jokes',Before I Let You in I need to make sure you're...,76.597,1.096649,17476,['“It’s black.”<br>“What?”<br>“Your phone scre...,b'Comedy',28425874,1735751


### Build Query

In [9]:
def create_custom_query(row):
    return (
        "You are a copywriter, create a 100 word summary of what this Youtube Short is about. "
        "Provide a neutral description. The summary should describe the overall atmosphere "
        "and pace of the video. It should also highlight important events from the video. "
        "Do not include any statements about a viewers response to the content or the overall "
        "viewing experience. Output the raw summary text.\n"
        "Title: {}\n"
        "Channel: {}\n"
        "Transcript: {}\n"
        "Comments: {}\n"
        "Category: {}\n"
    ).format(
        row['Video Title'], 
        row['Channel Title'], 
        row['Transcript'], 
        row['Top10 Comments'], 
        row['Category']
    )

In [10]:
# Apply the function to each row
df_shorts['Custom Query'] = df_shorts.apply(create_custom_query, axis=1)

### Query Huggingchat

In [11]:
import time

def get_chatbot_summary(row):
    #Cast to String for regex, since query returns Message object
    time.sleep(10)
    return str(chatbot.query(row['Custom Query']))

##### Batch Processing of Queries

to avoid potential errors like Rate Limiting or just the server being unavailable or your Internet connection being unavailable to destroy all the progress.

In [12]:
# Process the DataFrame in batches and update the DataFrame
def process_in_batches(dataframe, batch_size, output_csv):
    for start in range(0, len(dataframe), batch_size):
        end = min(start + batch_size, len(dataframe))
        batch = dataframe.iloc[start:end]

        with tqdm(total=len(batch), desc=f"Processing Batch {start}-{end}") as pbar:
            try:
                # Process each row and update the DataFrame
                for i, row in batch.iterrows():
                    dataframe.at[i, 'LLM Summary'] = get_chatbot_summary(row)
                    pbar.update(1)  # Update the batch progress bar

                # Write the current state of the DataFrame to the CSV file
                dataframe.to_csv(output_csv, index=False)

                print(f"Batch {start} to {end} processed successfully")
            
            except Exception as e:
                print(f"Error processing batch {start} to {end}: {e}")
                time.sleep(60)  # Sleep timer for rate limiting
                process_in_batches(dataframe.iloc[start:], batch_size, output_csv)  # Restart from the current batch
                break

In [ ]:
# Specify the output CSV file path
output_csv = "youtube_shorts_with_chatbot_summary_gdebus.csv"

# Process the DataFrame in batches of 20 and write to the CSV file
process_in_batches(df_shorts, 10, output_csv)

Processing Batch 0-10: 100%|███████████████████████████████████████████████████████████| 10/10 [03:30<00:00, 21.04s/it]


Batch 0 to 10 processed successfully


Processing Batch 10-20: 100%|██████████████████████████████████████████████████████████| 10/10 [03:23<00:00, 20.39s/it]


Batch 10 to 20 processed successfully


Processing Batch 20-30: 100%|██████████████████████████████████████████████████████████| 10/10 [03:30<00:00, 21.06s/it]


...


Processing Batch 330-340: 100%|████████████████████████████████████████████████████████| 10/10 [03:03<00:00, 18.39s/it]


Batch 330 to 340 processed successfully



Processing Batch 340-350:  30%|█████████████████                                        | 3/10 [00:57<02:14, 19.17s/it]Traceback (most recent call last):
  File "C:\Users\Gregor Debus\AppData\Local\Programs\Python\Python311\Lib\site-packages\hugchat\hugchat.py", line 686, in _stream_query
    obj = json.loads(res)
          ^^^^^^^^^^^^^^^
  File "C:\Users\Gregor Debus\AppData\Local\Programs\Python\Python311\Lib\json\__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gregor Debus\AppData\Local\Programs\Python\Python311\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gregor Debus\AppData\Local\Programs\Python\Python311\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (

Error processing batch 340 to 350: Failed to parse response: <!DOCTYPE html>




Processing Batch 0-10:   0%|                                                                    | 0/10 [00:00<?, ?it/s]

Processing Batch 0-10:  10%|██████                                                      | 1/10 [00:18<02:50, 18.96s/it]

Processing Batch 0-10:  20%|████████████                                                | 2/10 [00:37<02:27, 18.46s/it]

Processing Batch 0-10:  30%|██████████████████                                          | 3/10 [00:55<02:10, 18.63s/it]

Processing Batch 0-10:  40%|████████████████████████                                    | 4/10 [01:13<01:49, 18.27s/it]

Processing Batch 0-10:  50%|██████████████████████████████                              | 5/10 [01:32<01:33, 18.64s/it]

In [21]:
df_shorts.head()

,Video ID,Video Title,Channel Title,Transcript,Duration,Words per Second,Top10 Comments,Category,Custom Query,LLM Summary
0,l9_8_pDTmis,"b'Furthest Away From Me Wins $10,000'",MrBeast,whichever one of these three people can get th...,96.840,1.579926,"['Bro the guy at Disneyland was super smart, s...",Entertainment,"You are a copywriter, create a 100 word summar...","In this YouTube short, MrBeast challenges thr..."
1,QYEfTly0pTE,b'We lost contact with ATC over the ocean! #av...,Fly Me to the Fun™,this year we flew our Cessna 150 1800 miles fr...,110.080,1.435320,['Nice! <br><br>It’s always nice when the big ...,Travel & Events,"You are a copywriter, create a 100 word summar...",This YouTube short is about a flight from Atl...
2,jYJTPqU66IY,b'How to renovate your private jet',TheJetBusiness,I was going to say like what's a turnaround ti...,118.280,2.054447,['You know it’s a huge number when that was th...,Travel & Events,"You are a copywriter, create a 100 word summar...","Sure, here's a 100-word summary of the video ..."
3,dBsomKKHhtk,"b'When you finally meet the ""work wife""'",Viva La Dirt League,oh it's my w wife oh the the infamous W wife h...,120.879,1.886184,['Dude went from not realising he&#39;s marrie...,Gaming,"You are a copywriter, create a 100 word summar...","In this YouTube short, a man introduces his c..."
4,dTLYweJ08Tg,b'Courtside Kicks CASHES OUT on WHOLE TABLE of...,Courtside Kicks,yo what's good bro so you have a ton of dunks ...,59.679,1.524824,['After the video here is $20 for playing alon...,Entertainment,"You are a copywriter, create a 100 word summar...",This YouTube short features a person interest...


### Clean LLM Summaries 

This doesn't work 100% of the answers, since the answer varies a lot. I haven't found a pattern that is 100% reliable, but i would say this is "good enough"

In [22]:
import re

def clean_chatbot_response(text):
    # Regex pattern that accounts for variable spacing, punctuation, and case
    pattern = r"Sure[,!]? here'?s? (is |a |the )?100-word summary of .+:"
    # Replace the matched pattern with an empty string, ignoring case
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE).strip()
    return cleaned_text

In [23]:
df_shorts['LLM Summary'] = df_shorts['LLM Summary'].progress_apply(clean_chatbot_response)

100%|██████████| 20/20 [00:00<00:00, 19911.25it/s]


In [24]:
df_shorts.head()

,Video ID,Video Title,Channel Title,Transcript,Duration,Words per Second,Top10 Comments,Category,Custom Query,LLM Summary
0,l9_8_pDTmis,"b'Furthest Away From Me Wins $10,000'",MrBeast,whichever one of these three people can get th...,96.840,1.579926,"['Bro the guy at Disneyland was super smart, s...",Entertainment,"You are a copywriter, create a 100 word summar...","In this YouTube short, MrBeast challenges thre..."
1,QYEfTly0pTE,b'We lost contact with ATC over the ocean! #av...,Fly Me to the Fun™,this year we flew our Cessna 150 1800 miles fr...,110.080,1.435320,['Nice! <br><br>It’s always nice when the big ...,Travel & Events,"You are a copywriter, create a 100 word summar...",This YouTube short is about a flight from Atla...
2,jYJTPqU66IY,b'How to renovate your private jet',TheJetBusiness,I was going to say like what's a turnaround ti...,118.280,2.054447,['You know it’s a huge number when that was th...,Travel & Events,"You are a copywriter, create a 100 word summar...","In this video, a salesperson discusses the pro..."
3,dBsomKKHhtk,"b'When you finally meet the ""work wife""'",Viva La Dirt League,oh it's my w wife oh the the infamous W wife h...,120.879,1.886184,['Dude went from not realising he&#39;s marrie...,Gaming,"You are a copywriter, create a 100 word summar...","In this YouTube short, a man introduces his co..."
4,dTLYweJ08Tg,b'Courtside Kicks CASHES OUT on WHOLE TABLE of...,Courtside Kicks,yo what's good bro so you have a ton of dunks ...,59.679,1.524824,['After the video here is $20 for playing alon...,Entertainment,"You are a copywriter, create a 100 word summar...",This YouTube short features a person intereste...


In [2]:
df_shorts_curr = pd.read_csv("youtube_shorts_with_chatbot_summary.csv")
df_shorts_curr.concat(df_shorts).to_csv("youtube_shorts_with_chatbot_summary.csv", index=False)